In [1]:
meeting_id = 'd894ee56-d144-4cd5-8c2c-74de222a6b31'

In [49]:
meeting_id = '6cf0b989-5e9c-4a57-a058-8f6ac0ca8025'

In [50]:
from psql_helpers import get_meeting_by_id,async_session
import json
import pandas as pd

async with async_session() as session:
    meeting = await get_meeting_by_id(session, meeting_id)


In [51]:
df = pd.DataFrame(eval(meeting.transcript))
df.head()




,speaker,speaker_id,content,html_content,html_content_short,keywords,timestamp
0,Dmitriy Grankin,TBD,hey rick hey dimitri how are you,None,None,[],2024-10-03T18:06:40.070000Z
1,TBD,TBD,How was your day today?,None,None,[],2024-10-03T18:06:42.824000Z
2,TBD,TBD,"Yeah, long day.","Yeah, it was a long day.",,[],2024-10-03T18:06:45.204000Z
3,Dmitriy Grankin,TBD,So no traffic yet?,"So, <b>no traffic</b> yet?",no traffic,[],2024-10-03T18:06:49.864000Z
4,Dmitriy Grankin,TBD,No traffic yet.,No traffic yet.,,[],2024-10-03T18:06:51.844000Z


In [52]:
# Create a mask for speaker changes
speaker_change = (df['speaker'] != df['speaker'].shift())

# Create groups that reset when speaker changes or when same speaker speaks more than 3 times
group_id = (speaker_change | (df.groupby((speaker_change).cumsum())['speaker'].transform('size') > 5)).cumsum()

# Group the data and display results
grouped_df = df.groupby(group_id)

In [53]:
# Aggregate the groups by joining the text content
grouped_text = grouped_df.agg({
    'speaker': 'first',  # take first speaker name in group
    'content': ' '.join,    # join all text content
    'timestamp': 'first' # take first timestamp
})

In [54]:
grouped_df

In [55]:
from search import SearchAssistant
sa = SearchAssistant()
await sa.initialize()

In [56]:
grouped_text.head(10)

,speaker,content,timestamp
speaker,,,
1,Dmitriy Grankin,hey rick hey dimitri how are you,2024-10-03T18:06:40.070000Z
2,TBD,"How was your day today? Yeah, long day.",2024-10-03T18:06:42.824000Z
3,Dmitriy Grankin,"So no traffic yet? No traffic yet. Yeah, it takes a while.",2024-10-03T18:06:49.864000Z
4,Rick Tousseyn,Show me what you're trying to do. You want to... Yeah.,2024-10-03T18:06:58.016000Z
5,Dmitriy Grankin,"Yeah, what I was trying to do is that I just created this empty WordPress yeah and yeah understand that this I need to to choose a thing theme but i don't know um what i'm using is elementor so this one actually um is",2024-10-03T18:07:44.788000Z
6,Rick Tousseyn,sort of plugin that you install and it's and it's much better because you are able to create it in your your style and there's also like a big unity for this um what's the name what's the name yeah i'm gonna give you the link and one seconds This is what I recommend to everyone. OK.,2024-10-03T18:08:15.796000Z
7,Dmitriy Grankin,so basically yeah I want to set up this thing and blow it with articles and see how it goes yep,2024-10-03T18:08:40.338000Z
8,TBD,Elementor,2024-10-03T18:08:45.888000Z
9,Dmitriy Grankin,and Elementor is,2024-10-03T18:08:47.508000Z


In [63]:
# Import required libraries
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Load a pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings for all content
embeddings = model.encode(grouped_text['content'].values)

# Function to find similar sentences
def find_similar_sentences(query, embeddings, texts, top_k=20):
    # Encode the query
    query_embedding = model.encode([query])
    
    # Calculate similarities
    similarities = cosine_similarity(query_embedding, embeddings)[0]
    
    # Get top k similar indices
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    
    # Return results with similarities
    results = []
    for idx in top_indices:
        results.append({
            'text': texts.iloc[idx],
            'similarity': similarities[idx],
            'speaker': grouped_text.iloc[idx]['speaker'],
            'timestamp': grouped_text.iloc[idx]['timestamp']
        })
    
    return results

# Example usage:
# similar_segments = find_similar_sentences("your search query here", embeddings, grouped_text['content'])

In [64]:
pd.options.display.max_colwidth = 1000

In [65]:
q = "Surfer SEO"

In [66]:
r= find_similar_sentences(q, embeddings, grouped_text['content'])
pd.DataFrame(r)

,text,similarity,speaker,timestamp
0,"let's check this one so I'm gonna scan this one so they have so they have no schema you see they are they are not doing SEO very advanced but if you take like another website website um i don't know let's take let's take surfer seo for example yeah because i know that they are an seo company so let's have a look here in the in the validator when you test don't understand this don't understand this let me try another one schema validator you can also use this one google for This one, Google for developers, the schema testing tool. So rich results. You could make a great YouTube video out of it.",0.567605,Rick Tousseyn,2024-10-03T18:30:19.696000Z
1,"So I don't know if you ever saw like the articles from Surfer SEO, the AI generated articles.",0.551656,Rick Tousseyn,2024-10-03T19:15:46.674000Z
2,"Yeah, so for example, if you have a blog, it's interesting to have like this automated that it automatically links to other blogs from the same same topic yeah so you're creating a cluster so for example i show you like a cluster The Surfer SEO is actually quite nice for making this, but it's an expensive tool.",0.423354,Rick Tousseyn,2024-10-03T18:41:14.044000Z
3,pretty much required for good sale it's required for good SEO and if you want to,0.412420,Dmitriy Grankin,2024-10-03T18:32:52.172000Z
4,Search engine.,0.391747,Rick Tousseyn,2024-10-03T18:30:51.694000Z
5,"I think you're, you're in a good branch, you know, if you're part of of the next wave, then you really have a good business. You don't need people anymore to write because the AI can write better than a lot of copywriter, no mistake. no spelling mistakes it will write better maybe it will not know specific knowledge you know like very niche um maybe this is not possible but but for a lot of topics i think it can the second wave will come you're correct yeah i think that",0.391274,Rick Tousseyn,2024-10-03T19:14:34.884000Z
6,"going to increase even it's that was uh you know that was a first wave second And next waves will be massive, much, much bigger than this one. Then these ones we have, um,",0.354791,Dmitriy Grankin,2024-10-03T19:14:24.256000Z
7,so for example Asia marketing SEO in Singapore and so on you're creating all the ones that you have this map is really good for this but it will also tell you which ones you still need you which ones you still need okay so the uh purple ones that are con,0.351053,Rick Tousseyn,2024-10-03T18:42:33.572000Z
8,"This is what Google needs to understand your website and your product better. If you want to rank on number one, Google is going to pick the one that has the best information. And this is extra information. This is extra information. The user doesn't see this, by the way. This is only for the bot. But I can find it on the page.",0.349811,Rick Tousseyn,2024-10-03T18:32:11.916000Z
9,want to create many websites yeah exactly so yeah anyway so what is common is a massive wave of information online. And that's interesting how it's going on to how search engines search engine is going to deal with it oh yeah i think uh last year,0.334887,Dmitriy Grankin,2024-10-03T19:11:38.288000Z


In [62]:

global_results = await sa.search(q, user_id=va.user_id)
global_results.head(20)



/home/dima/playground/dev_multiassistant/back/search.py:114: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([main_df, speaker_df]).drop_duplicates(subset=columns).reset_index(drop=True)


,topic_name,speaker_name,summary,details,meeting_id,timestamp,score,vector_scores,exact_matches,source
0,Surfer SEO,Olga Mykhoparkina,Surfer SEO is a tool used for optimizing content based on SEO metrics.,"Olga mentions Surfer SEO's content editor, which helps in analyzing and improving the quality of articles based on competitor analysis.",d894ee56-d144-4cd5-8c2c-74de222a6b31,2024-10-03T09:00:51.040000,1.500000,"{'topic_name': 0.7000000500000001, 'summary': 0.59580548, 'details': 0.46483810000000003}","[topic_name, summary, details, topic_name, summary, details, topic_name, summary, details]",main
1,Surfer SEO,Rick Tousseyn,Surfer SEO is a tool that helps users optimize their content based on data-driven insights.,Rick discusses Surfer SEO in the context of creating content clusters and improving SEO strategies.,6cf0b989-5e9c-4a57-a058-8f6ac0ca8025,2024-10-03T18:06:40.070000,1.500000,"{'topic_name': 0.75000005, 'summary': 0.636142225, 'details': 0.5898206500000001}","[topic_name, summary, details, topic_name, summary, details, topic_name, summary, details]",main
2,Surfer SEO Utilization,Olga Mykhoparkina,Olga introduced Surfer SEO as a tool for optimizing content and improving article rankings on search engines.,"Surfer SEO provides a content editor that analyzes competitors and suggests improvements based on keyword usage, article length, and other factors. Olga suggested using it to enhance the quality of articles for better SEO performance.",d894ee56-d144-4cd5-8c2c-74de222a6b31,2024-10-03T09:00:51.040000,1.448214,"{'topic_name': 0.7241072, 'details': 0.59890422, 'summary': 0.560236425}","[topic_name, details, summary, topic_name, details, summary, topic_name, details, summary]",main
3,Surfer,Rick Tousseyn,Surfer is an SEO tool mentioned by Rick that helps optimize content based on keyword analysis and competition.,"Rick uses Surfer to benchmark content and improve SEO strategies, highlighting its features and benefits.",bdc05628-4278-4cda-9395-49906c348d7d,2024-10-01T09:19:05.583999,0.919880,"{'topic_name': 0.509939825, 'summary': 0.488020425, 'details': 0.463090875}",[],main
4,SEO,Rick Tousseyn,"SEO (Search Engine Optimization) is a key focus of the meeting, with discussions on tools and strategies to improve content ranking.","Rick, as the head of SEO at Team Lewis, shares insights on SEO tools and their importance in content optimization.",bdc05628-4278-4cda-9395-49906c348d7d,2024-10-01T09:19:05.583999,0.724903,{'topic_name': 0.7249034999999999},[],main
5,SEO,Rick Tousseyn,SEO (Search Engine Optimization) is the practice of optimizing websites to rank higher in search engine results.,The discussion revolves around using plugins like Rank Math and Yoast to improve SEO for Dmitriy's WordPress site.,6cf0b989-5e9c-4a57-a058-8f6ac0ca8025,2024-10-03T18:06:40.070000,0.724903,"{'topic_name': 0.7249034999999999, 'summary': 0.351805118}",[],main
6,SEO,Olga Mykhoparkina,"SEO stands for Search Engine Optimization, a strategy to improve website visibility in search engines.","The meeting covers various aspects of SEO, including content optimization, keyword usage, and the importance of internal linking.",d894ee56-d144-4cd5-8c2c-74de222a6b31,2024-10-03T09:00:51.040000,0.724903,"{'topic_name': 0.7249034999999999, 'summary': 0.38374009800000003}",[],main
7,SEO,Сергей,SEO (Search Engine Optimization) is a strategy discussed by Dmitry and Сергей to improve the visibility of VEX.AI's product in search engines.,They discuss the importance of optimizing the product's online presence to attract more users and improve search rankings.,0aee5d35-5e04-4b48-b428-6e2979aab55c,2024-09-25T13:28:55.560000,0.724903,{'topic_name': 0.7249034999999999},[],main
8,SEO,Dmitry Grankin,SEO is mentioned as a strategy to improve visibility and attract users.,The speakers discuss using SEO to extract specific cases and improve the product's online presence.,e2f3a86b-e94d-4967-8dad-89ac78a0617c,2024-09-13T10:17:35.355999,0.724903,"{'topic_name': 0.724903499999999

In [28]:
from vexa import VexaAPI
va = VexaAPI()
await va.get_user_info()
va.user_id



Vexa token: 3ae04e20124d40babc5107e658c666b6
User information retrieved successfully.


'ef7c085b-fdb5-4c94-b7b6-a61a3d04c210'